In [ ]:
# This is what I used to clean the sampled metadata file into something more workable for the knowledge graph.

In [7]:
import pandas as pd
import numpy as np
import re

In [14]:
%%time

# Load the CSV file
file_path = 'sampled_citations_metadata.csv'
df = pd.read_csv(file_path)

# 1. Clean up special characters (e.g., replace unwanted characters like `‚Äì` with actual dashes)
def clean_text(text):
    if isinstance(text, str):
        # Replace the special characters
        text = text.replace('‚Äì', '–').replace('Œ±', 'α').replace('…', '...')
        # Remove any remaining unwanted characters (non-printable characters)
        text = re.sub(r'[^\x00-\x7F]+', '', text)
        # Strip leading and trailing whitespaces
        text = text.strip()
    return text

# 2. Function to remove lists (and their contents) from a column
def omit_lists(value):
    if isinstance(value, list):
        return None  # Replace the list contents with None
    return value  # Return non-list values unchanged

# 3. Remove content inside square brackets (including brackets and leading space)
def remove_brackets(value):
    if isinstance(value, str):
        return re.sub(r'\s?\[.*?\]', '', value)  # Remove brackets and content inside
    return value

# 4. Parse the pub_date into pub_year, pub_month, and pub_day (handling m/d/yyyy, yyyy-mm, yyyy)
def parse_pub_date(pub_date):
    if pd.isna(pub_date):
        return None, None, None

    # Try to parse the date in various formats and extract year, month, and day
    try:
        # Check for full date (m/d/yyyy or other common formats)
        parsed = pd.to_datetime(pub_date, errors='coerce')
        if pd.notna(parsed):
            return parsed.year, parsed.month, parsed.day
    except:
        pass
    
    try:
        # Check for year and month (yyyy-mm format)
        parsed = pd.to_datetime(pub_date, errors='coerce', format='%Y-%m')
        if pd.notna(parsed):
            return parsed.year, parsed.month, None
    except:
        pass
    
    try:
        # Check for year only (yyyy format)
        parsed = pd.to_datetime(pub_date, errors='coerce', format='%Y')
        if pd.notna(parsed):
            return parsed.year, None, None
    except:
        pass

    # If parsing fails, return None for all
    return None, None, None

# Apply cleaning function to all string columns (standard text cleaning)
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].apply(clean_text)

# 5. Apply function to remove lists
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].apply(omit_lists)

# 6. Apply function to remove content in square brackets from author, venue, and publisher columns
for column in ['author', 'venue', 'publisher']:
    df[column] = df[column].apply(remove_brackets)

# 7. Handle missing values
# Fill missing values in 'page' column with blank strings instead of NaN
df['page'] = df['page'].fillna('')

# 8. Parse the pub_date into separate columns for year, month, and day
df[['pub_year', 'pub_month', 'pub_day']] = df['pub_date'].apply(parse_pub_date).apply(pd.Series)

# 9. Clean up author/editor columns: Convert them into lists or clean as necessary
# For example, if multiple authors are separated by ';', split them into lists
def clean_authors(authors_str):
    if isinstance(authors_str, str):
        authors = [author.strip() for author in authors_str.split(';')]
        return authors
    return []

df['author'] = df['author'].apply(clean_authors)
df['editor'] = df['editor'].apply(clean_authors)

# 10. Clean up venue and publisher columns (similar to author)
def clean_column(column_str):
    if isinstance(column_str, str):
        column_values = [value.strip() for value in column_str.split(';')]
        return column_values
    return []

df['venue'] = df['venue'].apply(clean_column)
df['publisher'] = df['publisher'].apply(clean_column)

# 11. Normalize text data in specific columns (optional, based on needs)
df['type'] = df['type'].str.lower()  # Convert the type column to lowercase

# 12. Convert list-type columns to strings before drop_duplicates
# Convert lists to strings to allow drop_duplicates to work
df = df.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

# 13. Remove any duplicate rows (if needed)
df = df.drop_duplicates()

# Save the cleaned DataFrame to a new CSV file
cleaned_file_path = 'sampled_citations_metadata_clean.csv'
df.to_csv(cleaned_file_path, index=False)

print(f"Cleaning completed. Cleaned file saved to {cleaned_file_path}")

Cleaning completed. Cleaned file saved to sampled_citations_metadata_clean.csv
CPU times: user 2.75 s, sys: 20.9 ms, total: 2.77 s
Wall time: 2.78 s


<timed exec>:106: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
